In [1]:
import os
import numpy as np
import torch

In [3]:
predicted_labels = np.array([[0, 1], [1, 2], [2, 5]])
true_labels = np.array([[0, 1], [2, 2], [1, 2], [3, 4], [4, 5],[2, 4], [1, 3], [-1, -1], [-1, -1]])

print(np.sum(true_labels, -1, keepdims=False) >= 0)
# print(true_labels[0])
true_labels = true_labels[np.sum(true_labels, -1, keepdims=False) > 0]

print(true_labels)

TP_edges = np.sum([np.any(np.all(e == true_labels, axis=1)) for e in predicted_labels])
TP_FP = len(predicted_labels)
TP_FN = len(true_labels)
precision = TP_edges / TP_FP
recall = TP_edges / TP_FN
print(TP_edges)
for e in predicted_labels:
    if np.any(np.all(e == true_labels, axis=1)):
        print(e)
# print([e in true_labels for e in predicted_labels])

# 计算 Precision 和 Recall
# precision, recall = calculate_precision_recall(predicted_labels, true_labels)

print("Precision:", precision)
print("Recall:", recall)

[ True  True  True  True  True  True  True False False]
[[0 1]
 [2 2]
 [1 2]
 [3 4]
 [4 5]
 [2 4]
 [1 3]]
2
[0 1]
[1 2]
Precision: 0.6666666666666666
Recall: 0.2857142857142857


In [4]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# 创建一个随机的权重矩阵
cost_matrix = np.array([[4, 1, 3],
                        [2, 0, 5],
                        [3, 2, 2]])

# 使用 linear_sum_assignment 求解最小权重匹配问题
row_indices, col_indices = linear_sum_assignment(cost_matrix)

print("Row indices:", row_indices)
print("Column indices:", col_indices)


Row indices: [0 1 2]
Column indices: [1 0 2]


In [5]:
import numpy as np
from scipy.spatial.distance import cdist

# Two sets of points with different lengths
point_set1 = np.array([[1, 2, 3], [7, 8, 9], [4, 5, 1], [7, 8, 9], [5,3,2], [1, 2, 4], [2, 5, 7], [1, 1, 1]])
point_set2 = np.array([[2, 3, 4], [5, 6, 4], [6, 7, 8],[-10, -10, -10], [-10, -10, -10]])
predicted_labels = np.array([[1, 2], [1, 5], [2, 4]])
true_labels = np.array([[0, 1], [0, 2], [1, 2], [3, 4], [4, 5],[2, 4], [1, 3], [-1, -1], [-1, -1]])
predicted_edges_indices = predicted_labels.flatten()
used_predicted_edges_indices = np.unique(predicted_edges_indices)
point_set1 = point_set1[used_predicted_edges_indices]
point_set2 = point_set2[np.sum(point_set2, -1, keepdims=False) > -10]
index_mapping = {old_index: new_index for new_index, old_index in enumerate(used_predicted_edges_indices)}
predicted_labels = np.vectorize(index_mapping.get)(predicted_labels)
# Calculate the distance matrix
distance_matrix = cdist(point_set1, point_set2)

print(distance_matrix)
row_indices, col_indices = linear_sum_assignment(distance_matrix)
print(row_indices, col_indices)
mask = distance_matrix[row_indices, col_indices]
# print(mask)
# print(mask < 6)
mask = mask < 6
print(mask)
result_dict = {key: value for key, value in zip(row_indices[mask], col_indices[mask])}
print(result_dict)
for i, _ in enumerate(predicted_labels):
    for j in range(2):
        predicted_labels[i, j] = result_dict[predicted_labels[i, j]] if predicted_labels[i, j] in result_dict else -1
    predicted_labels[i] = sorted(predicted_labels[i])
    # if value[0] in result_dict:
    #     print(result_dict[int(value[0])])
    #     predicted_labels[i, 0] = result_dict[value[0]]
    # else:
    #     predicted_labels[i, 0] = -1
        # print(result_dict[int(value[0])])
print(predicted_labels)
true_labels = true_labels[np.sum(true_labels, -1, keepdims=False) > 0]
TP_edges = np.sum([np.any(np.all(e == true_labels, axis=1)) for e in predicted_labels])
TP_FP = len(predicted_labels)
TP_FN = len(true_labels)
precision = TP_edges / TP_FP
recall = TP_edges / TP_FN
print("Precision:", precision)
print("Recall:", recall)
# print(true_labels)
print()
    # if result_dict[0]:
    #     print(value)
# print(row_indices[mask], col_indices[mask])
# print(point_set1[row_indices[mask]])
# print(point_set2[col_indices[mask]])
tp_corners = len(row_indices[mask])
tp_fp_corners = len(point_set1)
tp_fn_corners = len(point_set2)
print("corners Precision:", tp_corners/tp_fp_corners)
print("corners Recall:", tp_corners/tp_fn_corners)


[[8.66025404 5.74456265 1.73205081]
 [4.12310563 3.31662479 7.54983444]
 [3.60555128 3.60555128 7.28010989]
 [1.41421356 5.65685425 8.1240384 ]]
[0 1 3] [2 1 0]
[ True  True  True]
{0: 2, 1: 1, 3: 0}
[[ 1  2]
 [ 0  2]
 [-1  1]]
Precision: 0.6666666666666666
Recall: 0.2857142857142857

corners Precision: 0.75
corners Recall: 1.0


In [6]:
import numpy as np

# Two sets of 3D points
point_set1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
point_set2 = np.array([[1, 2, 3], [5, 6, 7], [8, 9, 10]])
print(cdist(point_set1, point_set2))

# Calculate the distance between each pair of points
distances = np.linalg.norm((point_set1 + 2) - (point_set2 + 2), axis=1)

print(distances)
print(np.sum(distances) / 3)

[[ 0.          6.92820323 12.12435565]
 [ 5.19615242  1.73205081  6.92820323]
 [10.39230485  3.46410162  1.73205081]]
[0.         1.73205081 1.73205081]
1.1547005383792515


In [7]:
import numpy as np

predicted_corners = np.array([[1, 2, 3], [7, 8, 9], [4, 5, 1], [7, 8, 9],[5,3,2]])
predicted_edges = np.array([[0, 2], [1, 2], [2, 4]])

# 找到所有在 predicted_edges 中出现的索引值
edge_indices = predicted_edges.flatten()

# 找到 predicted_corners 中出现在 predicted_edges 中的索引值
used_indices = np.unique(edge_indices)

# 根据 used_indices 来筛选 predicted_corners
predicted_corners = predicted_corners[used_indices]
print(predicted_corners)
# for i in range(len(predicted_corners)):
#     if i not in used_indices:
#         predicted_corners =
#         print(i)
# new_predicted_edges = np.delete(predicted_edges, 2, axis=0)



[[1 2 3]
 [7 8 9]
 [4 5 1]
 [5 3 2]]


In [2]:
import numpy as np
batch = {'batch_size': 1,
         'predicted_corners': np.array([[[1, 2, 3], [7, 8, 9], [4, 5, 1], [7, 8, 9], [5,3,2], [1, 2, 4], [2, 5, 7], [1, 1, 1]]]),
         'wf_vertices': np.array([[[2, 3, 4], [5, 6, 4], [6, 7, 8],[-10, -10, -10], [-10, -10, -10]]]),
         'predicted_edges': np.array([[[1, 2], [1, 5], [5, 6], [2, 4]]]),
         'wf_edges': np.array([[[0, 1], [0, 2], [1, 2], [3, 4], [4, 5],[2, 4], [1, 3], [-1, -1], [-1, -1]]]),
         'predicted_score': np.array([[0.8, 0.8, 0.2, 1]]),
           'centroid': np.array([[2, 2, 2]]),
         'max_distance': np.array([[1]])}

In [5]:
from ap_calculator import APCalculator
AP = APCalculator(3)
AP.compute_metrics(batch)
AP.output_accuracy()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 3

KeyboardInterrupt: 